# Toxic Comment Classification Challenge

#### STINTSY S13 Group 8
- VICENTE, Francheska Josefa
- VISTA, Sophia Danielle S.

## Problem Statement


## Description of the Dataset

## Requirements and Imports

### Datasets and Files


### Import

## Exploratory Data Analysis (EDA)

## Data Pre-Processing and Cleaning

## Feature Engineering

## Model Training

## Feature Selection and Hyperparameter tuning

## Model Selection

## Insights and Conclusions

## Testing the Model

## References